<a href="https://colab.research.google.com/github/th-spec/proyecto.tecnica/blob/main/originalperomejorelotro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================ CONFIGURACIÓN INICIAL ================
import os
import shutil
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pickle
import pandas as pd

# ================ BASE DE DATOS MEJORADA ================
# Estructura con múltiples imágenes por perro
if os.path.exists('pet_database.csv'):
    pet_database = pd.read_csv('pet_database.csv')
else:
    pet_database = pd.DataFrame(columns=['id', 'nombre', 'raza', 'dueño', 'imagenes_path'])

IMAGES_DIR = 'huellas_nasales/'
os.makedirs(IMAGES_DIR, exist_ok=True)

# ================ FUNCIÓN DE REGISTRO CORREGIDA ================
def register_pet():
    print("\n--- Registrar Nueva Mascota ---")
    print("Ahora puedes agregar MULTIPLES IMÁGENES por perro!")
    print("Ingresa las rutas una por una (deja vacío para terminar)\n")

    # Datos básicos del perro
    pet_id = input("ID único del perro: ")
    nombre = input("Nombre del perro: ")
    raza = input("Raza: ")
    dueño = input("Nombre del dueño: ")

    # Lista para almacenar rutas de imágenes
    imagenes = []

    # Bucle para múltiples imágenes
    while True:
        image_path = input(f"\nRuta de la imagen {len(imagenes)+1} (enter para terminar): ").strip()

        if not image_path:
            if len(imagenes) == 0:
                print("¡Debes agregar al menos 1 imagen!")
                continue
            break

        if not os.path.exists(image_path):
            print(f"¡Error: No se encontró {image_path}!")
            continue

        # Mover/copiar imagen al directorio de huellas
        filename = f"{pet_id}_{nombre}_{len(imagenes)+1}.jpg"
        save_path = os.path.join(IMAGES_DIR, filename)

        # Copiar la imagen (sin borrar el original)
        shutil.copy2(image_path, save_path)
        imagenes.append(save_path)

        print(f"Imagen {len(imagenes)} guardada como: {filename}")

    # Unir rutas con pipe (|) y guardar
    global pet_database
    new_entry = {
        'id': pet_id,
        'nombre': nombre,
        'raza': raza,
        'dueño': dueño,
        'imagenes_path': '|'.join(imagenes)  # Separador para múltiples rutas
    }

    pet_database = pd.concat([pet_database, pd.DataFrame([new_entry])], ignore_index=True)
    pet_database.to_csv('pet_database.csv', index=False)

    print(f"\n¡Registro exitoso! {nombre} tiene {len(imagenes)} imágenes registradas.")

# ================ FUNCIÓN PARA PREPROCESAR IMAGEN ================
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (200, 200))
    img = cv2.equalizeHist(img)
    return img.flatten() / 255.0

# ================ ENTRENAMIENTO MEJORADO ================
def train_model():
    # Verificar si hay suficientes datos
    if len(pet_database) < 3:
        print("Necesitas al menos 3 perros registrados")
        return

    total_images = sum(row['imagenes_path'].count('|') + 1 for _, row in pet_database.iterrows())
    if total_images < 10:
        print(f"Necesitas al menos 10 imágenes en total (actual: {total_images})")
        return

    X, y = [], []

    for _, row in pet_database.iterrows():
        for img_path in row['imagenes_path'].split('|'):
            try:
                features = preprocess_image(img_path)
                X.append(features)
                y.append(row['id'])
            except Exception as e:
                print(f"Error procesando {img_path}: {e}")

    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(X, y)

    with open('modelo_huellas.pkl', 'wb') as f:
        pickle.dump(model, f)

    print(f"✅ Modelo entrenado con {len(X)} imágenes de {len(pet_database)} perros")

# ================ FUNCIÓN PARA IDENTIFICAR MASCOTA ================
def identify_pet():
    # Aquí iría la lógica para identificar la mascota
    pass  # Implementa la lógica de identificación según tus necesidades

# ================ INTERFAZ PRINCIPAL ================
def main():
    # Cargar base de datos si existe
    global pet_database
    if os.path.exists('pet_database.csv'):
        pet_database = pd.read_csv('pet_database.csv')

    while True:
        print("\n=== SISTEMA DE IDENTIFICACIÓN CANINA ===")
        print("1. Registrar nueva mascota")
        print("2. Entrenar modelo de reconocimiento")
        print("3. Identificar mascota")
        print("4. Salir")

        choice = input("Selecciona una opción: ")

        if choice == '1':
            register_pet()
        elif choice == '2':
            train_model()
            print("Modelo entrenado y guardado!")
        elif choice == '3':
            identify_pet()
        elif choice == '4':
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida")

if __name__ == "__main__":
    main()




=== SISTEMA DE IDENTIFICACIÓN CANINA ===
1. Registrar nueva mascota
2. Entrenar modelo de reconocimiento
3. Identificar mascota
4. Salir
